## Data preprocessing

#### Loading GK matrix

In [ ]:
import rpy2.robjects as robjects
import numpy as np
import pandas as pd
# Load the Rdata file
robjects.r('load("../Geno/GKL.RData")')

# Get the list from the Rdata file
G_control_025 = robjects.r('GKL$Control$GK0.25')
G_control_15 = robjects.r('GKL$Control$GK0.25')
# G_stress = robjects.r('GL$Stress')

# # Convert into numpy array
G_con_025 = pd.DataFrame(np.reshape(G_control_025, (G_control_025.nrow, G_control_025.ncol)), columns = G_control_025.colnames, index = G_control_025.rownames)
G_con_15 = pd.DataFrame(np.reshape(G_control_15, (G_control_15.nrow, G_control_15.ncol)), columns = G_control_15.colnames, index = G_control_15.rownames)

### Loading Metabolites

In [ ]:
import os
metL = []
for i in range(1, 31):
    metL.append(pd.read_csv('../Met/CrossValidation/cv_' + str(i) + '/met_cv_' + str(i) + '.csv'))

In [ ]:
def met_read(met_cv1):

    # met_cv1 = pd.read_csv('../Met/CrossValidation/cv_1/met_cv_1.csv')
    met_con_cv1_train = met_cv1.query("Treatment=='Control' and set == 'train'")
    met_con_cv1_test = met_cv1.query("Treatment=='Control' and set == 'test'")
    train_id = np.array(met_con_cv1_train.NSFTV_ID)
    test_id = np.array(met_con_cv1_test.NSFTV_ID)

    G_con_train025 = G_con_025.loc[train_id, ] #155*155
    G_con_test025 = G_con_025.loc[test_id, ] #38*38
    G_con_train15 = G_con_15.loc[train_id, ] #155*155
    G_con_test15 = G_con_15.loc[test_id, ]
    # Convert pd.dataframe into np.array
    X_train025 = np.array(G_con_train025)
    X_test025 = np.array(G_con_test025)
    X_train15 = np.array(G_con_train15)
    X_test15 = np.array(G_con_test15)
    Y_train = np.array(met_con_cv1_train.iloc[:,2]) #remove Treatment, set columns 
                                                    #just keep metabolie a1
    Y_test = np.array(met_con_cv1_test.iloc[:,2]) #remove Treatment, set columns 
                                                #just keep metabolie a1
    # print('X_train shape is:', X_train.shape)
    # print('X_test shape is:', X_test.shape)
    # print('Y_train shape is:', Y_train.shape)
    # print('Y_test shape is:', Y_test.shape)

    return X_train025, X_test025, X_train15, X_test15, Y_train, Y_test

In [ ]:
X_train025, X_test025, X_train15, X_test15, Y_train, Y_test = met_read(metL[0])

## Y-net for CNN

In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, GlobalAveragePooling1D, GlobalMaxPool1D, MaxPool1D, Dropout, Input, concatenate, Activation
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import random
import numpy as np
import tensorflow as tf

In [ ]:
def model_build(input_shape, batch_size,kernel_size,dropout,n_filters):
    
    # left branch of Y network
    left_inputs = Input(shape=input_shape)
    x = left_inputs
    filters = n_filters
    # 3 layers of Conv2D-Dropout-MaxPooling2D
    # number of filters doubles after each layer (32-64-128)
    for i in range(3):
        x = Conv1D(filters=filters,
                  kernel_size=kernel_size,
                  padding='same',
                  activation='relu')(x)
        x = Dropout(dropout)(x)
        x = MaxPooling1D()(x)
        filters *= 2

    # right branch of Y network
    right_inputs = Input(shape=input_shape)
    y = right_inputs
    filters = n_filters
    # 3 layers of Conv2D-Dropout-MaxPooling2D
    # number of filters doubles after each layer (32-64-128)
    for i in range(3):
        y = Conv1D(filters=filters,
                  kernel_size=kernel_size, #3*3
                  padding='same', #padding zeros
                  activation='relu',
                  dilation_rate=2)(y)
        y = Dropout(dropout)(y)
        y = MaxPooling1D()(y) #pool_size=(2, 2) default
        filters *= 2

    # merge left and right branches outputs
    z = concatenate([x, y])
    # feature maps to vector in preparation to connecting to Dense layer
    z = Flatten()(z)
    z = Dropout(dropout)(z)
    outputs = Dense(num_labels, activation='softmax')(z)

    # build the model in functional API
    model = Model([left_inputs, right_inputs], outputs)
    # verify the model using graph
    # plot_model(model, to_file='cnn-y-network.png', show_shapes=True)
    # verify the model using layer text description
    print(model.summary())
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    

    return model

In [ ]:
# def CNN(X_train025, X_test025, X_train15, X_test15, Y_train, Y_test):

    seed=616

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = Sequential()
    model.add(Conv1D(32, 8, activation="relu", input_shape=(193,1)))
    # model.add(Conv1D(64, 8, activation="relu"))
    # model.add(Conv1D(128, 8, activation="relu"))
    # model.add(Conv1D(512, 8, activation="relu"))
    # model.add(Conv1D(32, 8, activation="relu"))
    # model.add(GlobalAveragePooling1D())
    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dense(1))
    opt = keras.optimizers.legacy.Adam(learning_rate=1e-5)
    model.compile(loss="mse", optimizer=opt, metrics=['mse'])
    # model.summary()


In [ ]:
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    history_cnn = model.fit(X_train, Y_train, 
                            batch_size=12,
                            epochs=200, 
                            verbose=0,
                            validation_data=(X_test, Y_test),
                            callbacks=[early_stopping])

    Y_pred = model.predict(X_test).reshape(38)

    print("MSE:", mean_squared_error(Y_test, Y_pred))
    print("Corr:", np.corrcoef(Y_test, Y_pred)[0,1])


    print('############################ \n')
    
    # return history_cnn

In [ ]:
for i in range(0, 1):
    print("NOW is running met #", i+1)
    X_train, X_test, Y_train, Y_test = met_read(metL[i])
    history_cnn = CNN(X_train, X_test, Y_train, Y_test)

In [ ]:
plt.plot(history_cnn.history['loss'], label='loss')
plt.plot(history_cnn.history['val_loss'], label = 'val_loss')

plt.xlabel('Epoch')
plt.ylabel('LOSS')
plt.legend(loc='upper right')
